In [1]:
# !conda install Elasticsearch
from elasticsearch import Elasticsearch 
from elasticsearch import helpers
import pandas as pd

In [2]:
#connect to elasticsearch server
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [3]:
import requests
# get info about the cluster
res = requests.get('http://localhost:9200')
print(res.content)

b'{\n  "name" : "0d78a33bf061",\n  "cluster_name" : "elasticsearch",\n  "cluster_uuid" : "4m-q-nETQTSgdtLNyUWDRQ",\n  "version" : {\n    "number" : "7.16.2",\n    "build_flavor" : "default",\n    "build_type" : "tar",\n    "build_hash" : "2b937c44140b6559905130a8650c64dbd0879cfb",\n    "build_date" : "2021-12-18T19:42:46.604893745Z",\n    "build_snapshot" : false,\n    "lucene_version" : "8.10.1",\n    "minimum_wire_compatibility_version" : "6.8.0",\n    "minimum_index_compatibility_version" : "6.0.0-beta1"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [4]:
import json
r = requests.get('http://localhost:9200')
i = 1

# add data from star wars api to the index
while r.status_code == 200:
    r = requests.get('https://swapi.dev/api/people/'+ str(i))
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1

print(i)

# we automatically created an index "sw" and a "doc_type" with de indexing command (automatic mapping)

C:\Users\andrea.cortesi\Miniconda3\envs\gym\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
C:\Users\andrea.cortesi\Miniconda3\envs\gym\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


18


In [5]:
# request : get the document with id=5
es.get(index='sw', doc_type='people', id=5)

C:\Users\andrea.cortesi\Miniconda3\envs\gym\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: [types removal] Specifying types in document get requests is deprecated, use the /{index}/_doc/{id} endpoint instead.
  warnings.warn(message, category=ElasticsearchWarning)


{'_index': 'sw',
 '_type': 'people',
 '_id': '5',
 '_version': 1,
 '_seq_no': 4,
 '_primary_term': 1,
 'found': True,
 '_source': {'name': 'Leia Organa',
  'height': '150',
  'mass': '49',
  'hair_color': 'brown',
  'skin_color': 'light',
  'eye_color': 'brown',
  'birth_year': '19BBY',
  'gender': 'female',
  'homeworld': 'https://swapi.dev/api/planets/2/',
  'films': ['https://swapi.dev/api/films/1/',
   'https://swapi.dev/api/films/2/',
   'https://swapi.dev/api/films/3/',
   'https://swapi.dev/api/films/6/'],
  'species': [],
  'vehicles': ['https://swapi.dev/api/vehicles/30/'],
  'starships': [],
  'created': '2014-12-10T15:20:09.791000Z',
  'edited': '2014-12-20T21:17:50.315000Z',
  'url': 'https://swapi.dev/api/people/5/'}}

In [6]:
r = requests.get('http://localhost:9201')
i = 18
while r.status_code == 200:
    r = requests.get('https://swapi.dev/api/people/'+ str(i))
    es.index(index='sw', doc_type='people', id=i,     body=json.loads(r.content))
    i=i+1

ConnectionError: HTTPConnectionPool(host='localhost', port=9201): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001AF9671F3D0>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))

In [7]:
es.search(index="sw", body={"query": {"match": {'name':'Darth Vader'}}})


{'took': 274,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 4.9164443,
  'hits': [{'_index': 'sw',
    '_type': 'people',
    '_id': '4',
    '_score': 4.9164443,
    '_source': {'name': 'Darth Vader',
     'height': '202',
     'mass': '136',
     'hair_color': 'none',
     'skin_color': 'white',
     'eye_color': 'yellow',
     'birth_year': '41.9BBY',
     'gender': 'male',
     'homeworld': 'https://swapi.dev/api/planets/1/',
     'films': ['https://swapi.dev/api/films/1/',
      'https://swapi.dev/api/films/2/',
      'https://swapi.dev/api/films/3/',
      'https://swapi.dev/api/films/6/'],
     'species': [],
     'vehicles': [],
     'starships': ['https://swapi.dev/api/starships/13/'],
     'created': '2014-12-10T15:18:20.704000Z',
     'edited': '2014-12-20T21:17:50.313000Z',
     'url': 'https://swapi.dev/api/people/4/'}}]}}

In [8]:
es.search(index="sw", body={"query": {"prefix": {'name':'dar'}}})


{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'sw',
    '_type': 'people',
    '_id': '4',
    '_score': 1.0,
    '_source': {'name': 'Darth Vader',
     'height': '202',
     'mass': '136',
     'hair_color': 'none',
     'skin_color': 'white',
     'eye_color': 'yellow',
     'birth_year': '41.9BBY',
     'gender': 'male',
     'homeworld': 'https://swapi.dev/api/planets/1/',
     'films': ['https://swapi.dev/api/films/1/',
      'https://swapi.dev/api/films/2/',
      'https://swapi.dev/api/films/3/',
      'https://swapi.dev/api/films/6/'],
     'species': [],
     'vehicles': [],
     'starships': ['https://swapi.dev/api/starships/13/'],
     'created': '2014-12-10T15:18:20.704000Z',
     'edited': '2014-12-20T21:17:50.313000Z',
     'url': 'https://swapi.dev/api/people/4/'}},
   {'_index': 'sw',
    '_type': 'people',
    '_id': '9'

In [9]:
es.search(index="sw", body={"query": {"prefix" : { "name" : "lu" }}})


{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'sw',
    '_type': 'people',
    '_id': '1',
    '_score': 1.0,
    '_source': {'name': 'Luke Skywalker',
     'height': '172',
     'mass': '77',
     'hair_color': 'blond',
     'skin_color': 'fair',
     'eye_color': 'blue',
     'birth_year': '19BBY',
     'gender': 'male',
     'homeworld': 'https://swapi.dev/api/planets/1/',
     'films': ['https://swapi.dev/api/films/1/',
      'https://swapi.dev/api/films/2/',
      'https://swapi.dev/api/films/3/',
      'https://swapi.dev/api/films/6/'],
     'species': [],
     'vehicles': ['https://swapi.dev/api/vehicles/14/',
      'https://swapi.dev/api/vehicles/30/'],
     'starships': ['https://swapi.dev/api/starships/12/',
      'https://swapi.dev/api/starships/22/'],
     'created': '2014-12-09T13:50:51.644000Z',
     'edited': '2014-12-20T21

In [10]:
# es.search(index="sw", body={"query":
# {"fuzzy_like_this_field" : { "name" :
# {"like_text": "jaba", "max_query_terms":5}}}})

es.search(index="sw", body={"query": {"fuzzy": {'name':'jaba'}}})

{'took': 14,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.5353181,
  'hits': [{'_index': 'sw',
    '_type': 'people',
    '_id': '16',
    '_score': 1.5353181,
    '_source': {'name': 'Jabba Desilijic Tiure',
     'height': '175',
     'mass': '1,358',
     'hair_color': 'n/a',
     'skin_color': 'green-tan, brown',
     'eye_color': 'orange',
     'birth_year': '600BBY',
     'gender': 'hermaphrodite',
     'homeworld': 'https://swapi.dev/api/planets/24/',
     'films': ['https://swapi.dev/api/films/1/',
      'https://swapi.dev/api/films/3/',
      'https://swapi.dev/api/films/4/'],
     'species': ['https://swapi.dev/api/species/5/'],
     'vehicles': [],
     'starships': [],
     'created': '2014-12-10T17:11:31.638000Z',
     'edited': '2014-12-20T21:17:50.338000Z',
     'url': 'https://swapi.dev/api/people/16/'}}]}}

## Biblio

* https://betterprogramming.pub/quick-start-elasticsearch-with-python-7756ea45d815

* https://tryolabs.com/blog/2015/02/17/python-elasticsearch-first-steps

* https://swapi.dev/

* https://www.hackdeploy.com/python-elasticsearch-getting-started-guide/